# Transfer Learning Task

The goal of this task is to illustrate how pretrained neural networks can be used for soving different classification problems.

## Data

We will use the [Cifar 10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). It consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](images/cifar10.png)



In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(y_train.shape)

Prepare data

In [0]:
from keras.utils import np_utils
import numpy as np

n_classes = 10

y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

Import pretrained model

In [6]:
from keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
for layer in base_model.layers[:-4]:
    layer.trainable = False

#from keras.applications.resnet50 import ResNet50
#base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

W0716 09:14:01.487159 140186557073280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0716 09:14:01.503580 140186557073280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0716 09:14:01.551055 140186557073280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 25s 0us/step


W0716 09:14:28.555120 140186557073280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0716 09:14:28.556808 140186557073280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



Add new top layers

In [7]:
from keras import models, layers
    
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


W0716 09:14:36.102345 140186557073280 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Compile the model

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0716 09:15:13.903892 140186557073280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [9]:
model.fit(x_train, y_train,
          batch_size = 512, epochs = 10, verbose=1,
          validation_data=(x_test, y_test))

W0716 09:16:10.717417 140186557073280 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 29s 590us/step - loss: 1.4828 - acc: 0.4579 - val_loss: 0.9756 - val_acc: 0.6612
Epoch 2/10
50000/50000 [==============================] - 20s 392us/step - loss: 0.8802 - acc: 0.7000 - val_loss: 0.8370 - val_acc: 0.7128
Epoch 3/10
50000/50000 [==============================] - 20s 391us/step - loss: 0.7319 - acc: 0.7487 - val_loss: 0.7746 - val_acc: 0.7363
Epoch 4/10
50000/50000 [==============================] - 20s 391us/step - loss: 0.6333 - acc: 0.7843 - val_loss: 0.7465 - val_acc: 0.7458
Epoch 5/10
50000/50000 [==============================] - 20s 392us/step - loss: 0.5492 - acc: 0.8121 - val_loss: 0.7664 - val_acc: 0.7497
Epoch 6/10
50000/50000 [==============================] - 20s 391us/step - loss: 0.4752 - acc: 0.8365 - val_loss: 0.7620 - val_acc: 0.7521
Epoch 7/10
50000/50000 [==============================] - 20s 392us/step - loss: 0.4113 - acc: 0.8595 - val_loss: 0.

## Evaluate the model

First we need to convert probability vectors to class indices.

In [11]:
y_pred = model.predict(x_test)

print(y_pred.shape)

(10000, 10)


In [12]:
import numpy as np

y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
print(y_pred_class.shape)

(10000,)


In [13]:
from sklearn import metrics
from sklearn.metrics import accuracy_score


print ("Test accuracy: {:.4f}".format(accuracy_score(y_test_class, y_pred_class)))
print ()
print(metrics.classification_report(y_test_class, y_pred_class, digits=4))

Test accuracy: 0.7475

              precision    recall  f1-score   support

           0     0.8288    0.7600    0.7929      1000
           1     0.8729    0.7900    0.8294      1000
           2     0.6586    0.7370    0.6956      1000
           3     0.6147    0.5010    0.5521      1000
           4     0.7311    0.6770    0.7030      1000
           5     0.6971    0.6190    0.6557      1000
           6     0.6816    0.8670    0.7632      1000
           7     0.8153    0.8080    0.8117      1000
           8     0.7918    0.9050    0.8446      1000
           9     0.7920    0.8110    0.8014      1000

    accuracy                         0.7475     10000
   macro avg     0.7484    0.7475    0.7450     10000
weighted avg     0.7484    0.7475    0.7450     10000



In [14]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))

[[760  16  62  14  20   4  17  12  75  20]
 [ 15 790   4  11   1   5  14   2  49 109]
 [ 28   3 737  34  52  24  84  15  16   7]
 [ 14   3  79 501  46 155 122  33  24  23]
 [ 16   3  89  43 677  26  73  54  12   7]
 [  9   3  61 135  36 619  66  50   9  12]
 [  4   2  44  25  24  13 867   5  10   6]
 [ 11   3  29  32  59  32  11 808   3  12]
 [ 33  14  10   4   6   2   6   3 905  17]
 [ 27  68   4  16   5   8  12   9  40 811]]
